# Data Ingestion and Cleaning

This notebook loads raw CSV datasets, cleans column names, normalizes city names, parses dates, and handles missing values.

## Data Sources
- `weather_india.csv`: Weather data (rainfall, temperature)
- `traffic_india.csv`: Traffic congestion levels
- `demand_india.csv`: Demand index for quick-commerce orders


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

# Add src to path for config imports
sys.path.append('../src')

# Set up paths
RAW_DATA_DIR = Path('../data/raw')
PROCESSED_DATA_DIR = Path('../data/processed')

# Create processed directory if it doesn't exist
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Raw data directory: {RAW_DATA_DIR}")
print(f"Processed data directory: {PROCESSED_DATA_DIR}")


## Load Raw Data


In [ ]:
# Load all raw CSV files
weather_df = pd.read_csv(RAW_DATA_DIR / 'weather_india.csv')
traffic_df = pd.read_csv(RAW_DATA_DIR / 'traffic_india.csv')
demand_df = pd.read_csv(RAW_DATA_DIR / 'demand_india.csv')

print("Weather data shape:", weather_df.shape)
print("Traffic data shape:", traffic_df.shape)
print("Demand data shape:", demand_df.shape)

print("\nWeather columns:", weather_df.columns.tolist())
print("Traffic columns:", traffic_df.columns.tolist())
print("Demand columns:", demand_df.columns.tolist())


## Clean Column Names


In [ ]:
# Standardize column names: strip whitespace, lowercase, replace spaces with underscores
def clean_column_names(df):
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df

weather_df = clean_column_names(weather_df)
traffic_df = clean_column_names(traffic_df)
demand_df = clean_column_names(demand_df)

print("Cleaned column names:")
print("Weather:", weather_df.columns.tolist())
print("Traffic:", traffic_df.columns.tolist())
print("Demand:", demand_df.columns.tolist())


## Normalize City Names


In [ ]:
# Normalize city names: title case, strip whitespace
def normalize_city_names(df, city_col='city'):
    df = df.copy()
    df[city_col] = df[city_col].str.strip().str.title()
    return df

weather_df = normalize_city_names(weather_df)
traffic_df = normalize_city_names(traffic_df)
demand_df = normalize_city_names(demand_df)

print("Unique cities in each dataset:")
print("Weather:", sorted(weather_df['city'].unique()))
print("Traffic:", sorted(traffic_df['city'].unique()))
print("Demand:", sorted(demand_df['city'].unique()))


## Parse Dates


In [ ]:
# Parse date columns
weather_df['date'] = pd.to_datetime(weather_df['date'])
traffic_df['date'] = pd.to_datetime(traffic_df['date'])
demand_df['date'] = pd.to_datetime(demand_df['date'])

print("Date ranges:")
print(f"Weather: {weather_df['date'].min()} to {weather_df['date'].max()}")
print(f"Traffic: {traffic_df['date'].min()} to {traffic_df['date'].max()}")
print(f"Demand: {demand_df['date'].min()} to {demand_df['date'].max()}")

print("\nDate column dtypes:")
print(f"Weather date dtype: {weather_df['date'].dtype}")
print(f"Traffic date dtype: {traffic_df['date'].dtype}")
print(f"Demand date dtype: {demand_df['date'].dtype}")


## Handle Missing Values


In [ ]:
# Check for missing values
print("Missing values before cleaning:")
print("\nWeather:")
print(weather_df.isnull().sum())
print("\nTraffic:")
print(traffic_df.isnull().sum())
print("\nDemand:")
print(demand_df.isnull().sum())


In [ ]:
# Handle missing values
# For weather: forward fill rainfall, interpolate temperature
# For traffic: forward fill congestion
# For demand: forward fill demand_index

def handle_missing_values(df, dataset_name):
    df = df.copy()
    
    # Sort by date and city to ensure proper forward fill
    df = df.sort_values(['city', 'date'])
    
    if dataset_name == 'weather':
        # Forward fill rainfall (assume same as previous day if missing)
        df['rainfall_mm'] = df.groupby('city')['rainfall_mm'].ffill().fillna(0)
        # Interpolate temperature (linear interpolation)
        df['temperature'] = df.groupby('city')['temperature'].interpolate(method='linear')
        # If still missing, forward fill
        df['temperature'] = df.groupby('city')['temperature'].ffill()
        
    elif dataset_name == 'traffic':
        # Forward fill congestion level
        df['congestion_level'] = df.groupby('city')['congestion_level'].ffill()
        # If still missing, use median by city
        city_medians = df.groupby('city')['congestion_level'].transform('median')
        df['congestion_level'] = df['congestion_level'].fillna(city_medians)
        
    elif dataset_name == 'demand':
        # Forward fill demand index
        df['demand_index'] = df.groupby('city')['demand_index'].ffill()
        # If still missing, use median by city
        city_medians = df.groupby('city')['demand_index'].transform('median')
        df['demand_index'] = df['demand_index'].fillna(city_medians)
    
    return df

weather_df = handle_missing_values(weather_df, 'weather')
traffic_df = handle_missing_values(traffic_df, 'traffic')
demand_df = handle_missing_values(demand_df, 'demand')

print("Missing values after cleaning:")
print("\nWeather:")
print(weather_df.isnull().sum())
print("\nTraffic:")
print(traffic_df.isnull().sum())
print("\nDemand:")
print(demand_df.isnull().sum())


## Validate Data Quality


In [ ]:
# Validate data ranges and types
print("Weather data validation:")
print(f"Rainfall range: {weather_df['rainfall_mm'].min():.2f} - {weather_df['rainfall_mm'].max():.2f} mm")
print(f"Temperature range: {weather_df['temperature'].min():.2f} - {weather_df['temperature'].max():.2f} °C")

print("\nTraffic data validation:")
print(f"Congestion range: {traffic_df['congestion_level'].min():.2f} - {traffic_df['congestion_level'].max():.2f}")

print("\nDemand data validation:")
print(f"Demand index range: {demand_df['demand_index'].min():.2f} - {demand_df['demand_index'].max():.2f}")

# Check for duplicates
print("\nDuplicate check (date, city combinations):")
print(f"Weather duplicates: {weather_df.duplicated(subset=['date', 'city']).sum()}")
print(f"Traffic duplicates: {traffic_df.duplicated(subset=['date', 'city']).sum()}")
print(f"Demand duplicates: {demand_df.duplicated(subset=['date', 'city']).sum()}")


## Save Cleaned Data


In [ ]:
# Save cleaned datasets
weather_df.to_csv(PROCESSED_DATA_DIR / 'weather_cleaned.csv', index=False)
traffic_df.to_csv(PROCESSED_DATA_DIR / 'traffic_cleaned.csv', index=False)
demand_df.to_csv(PROCESSED_DATA_DIR / 'demand_cleaned.csv', index=False)

print("Cleaned data saved to:")
print(f"  - {PROCESSED_DATA_DIR / 'weather_cleaned.csv'}")
print(f"  - {PROCESSED_DATA_DIR / 'traffic_cleaned.csv'}")
print(f"  - {PROCESSED_DATA_DIR / 'demand_cleaned.csv'}")

print("\nData ingestion complete!")
